In [2]:
import pandas as pd 
import numpy as np 

## Index objects

In [28]:
df = pd.DataFrame(np.random.randn(5, 3), 
columns=['A', 'B', 'C'])
df

,A,B,C
0,1.445456,0.622485,-1.255156
1,0.575118,-1.104595,1.407740
2,-0.137163,0.582018,-1.380512
3,-1.260781,0.554123,-0.316706
4,-0.967927,-1.069614,-0.226388


In [29]:
df.columns

Index(['A', 'B', 'C'], dtype='object')

In [30]:
df.index

RangeIndex(start=0, stop=5, step=1)

## Index objects: creating an index object

In [93]:
pd.Index(['A','B','C'])


Index(['A', 'B', 'C'], dtype='object')

In [94]:
pd.Index(['A','B','C','A'])


Index(['A', 'B', 'C', 'A'], dtype='object')

In [98]:
pd.Index(range(5))

RangeIndex(start=0, stop=5, step=1)

In [97]:
pd.Index(list(range(5)))

Index([0, 1, 2, 3, 4], dtype='int64')

In [99]:
a = pd.Index(['c', 'b', 'a'])
b = pd.Index(['c', 'e', 'd'])

In [100]:
a.union(b)

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [101]:
a.intersection(b)

Index(['c'], dtype='object')

In [102]:
c = pd.Index([1.0, 1.5, 2.0])
d = pd.Index(range(5))
c.union(d)

Index([0.0, 1.0, 1.5, 2.0, 3.0, 4.0], dtype='float64')

In [103]:
a.difference(b)

Index(['a', 'b'], dtype='object')

In [104]:
a.symmetric_difference(b)

Index(['a', 'b', 'd', 'e'], dtype='object')

In [105]:
e = pd.Index(["A","B","C"])
f = pd.Index(range(5))
e.union(f)

Index(['A', 'B', 'C', 0, 1, 2, 3, 4], dtype='object')

In [106]:
df = pd.DataFrame( np.random.randn(3, 3),
                  index=pd.Index(['x','y','z'],name="rows"),
                  columns=pd.Index(['A', 'B', 'C'],name="cols"))
df.columns

Index(['A', 'B', 'C'], dtype='object', name='cols')

In [107]:
df.index

Index(['x', 'y', 'z'], dtype='object', name='rows')

In [108]:
df.columns.rename("m")


Index(['A', 'B', 'C'], dtype='object', name='m')

In [110]:
df.index.set_names("n")


Index(['x', 'y', 'z'], dtype='object', name='n')

In [111]:
df.columns.name = "o"
df.index.rename("p", inplace=
True)
df

o,A,B,C
p,,,
x,-0.736756,-2.391423,0.739519
y,0.061929,-0.272410,1.131421
z,0.189589,0.443302,1.455106


## Indexes and missing values

In [112]:
pd.Index([1,2,3,np.nan,5])

Index([1.0, 2.0, 3.0, nan, 5.0], dtype='float64')

In [113]:
pd.Index(["A","B",np.nan,"D"])

Index(['A', 'B', nan, 'D'], dtype='object')

In [114]:
pd.Index([1,2,3,np.nan,5]).fillna(0)

Index([1.0, 2.0, 3.0, 0.0, 5.0], dtype='float64')

In [115]:
pd.Index(["A","B",np.nan,"D"]).fillna("Z")

Index(['A', 'B', 'Z', 'D'], dtype='object')

## changing a DataFrame’s index

In [118]:
data = pd.DataFrame([["bar", "one", "z", 1], 
                    ["bar", "two", "y", 2], 
                    ["foo", "one", "x", 3], 
                    ["foo", "two", "w", 4]], columns=['a', 'b', 'c', 'd'])
data

,a,b,c,d
0,bar,one,z,1
1,bar,two,y,2
2,foo,one,x,3
3,foo,two,w,4


In [119]:
data.set_index('a')

,b,c,d
a,,,
bar,one,z,1
bar,two,y,2
foo,one,x,3
foo,two,w,4


In [121]:
data.set_index('c', drop=False)

,a,b,c,d
c,,,,
z,bar,one,z,1
y,bar,two,y,2
x,foo,one,x,3
w,foo,two,w,4


In [122]:
data.reindex(["w","x","y","z"])

,a,b,c,d
w,NaN,NaN,NaN,NaN
x,NaN,NaN,NaN,NaN
y,NaN,NaN,NaN,NaN
z,NaN,NaN,NaN,NaN


In [123]:
data.reindex(range(5,-1,-1))

,a,b,c,d
5,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
3,foo,two,w,4.0
2,foo,one,x,3.0
1,bar,two,y,2.0
0,bar,one,z,1.0


In [124]:
data.set_index('a').reset_index()

,a,b,c,d
0,bar,one,z,1
1,bar,two,y,2
2,foo,one,x,3
3,foo,two,w,4


In [125]:
data.set_index('c').reset_index(drop=True)

,a,b,d
0,bar,one,1
1,bar,two,2
2,foo,one,3
3,foo,two,4


In [126]:
data.reindex(columns = ["a","b","c","d","e"])

,a,b,c,d,e
0,bar,one,z,1,NaN
1,bar,two,y,2,NaN
2,foo,one,x,3,NaN
3,foo,two,w,4,NaN


In [127]:
data.index = ["w","x","y","z"]

In [128]:
data 

,a,b,c,d
w,bar,one,z,1
x,bar,two,y,2
y,foo,one,x,3
z,foo,two,w,4


## Multi-indexes

In [129]:
tuples = [('A','x'), ('A','y'),('B','x'), ('B','y'),('C','x'), ('C','y')]
pd.MultiIndex.from_tuples(tuples, names=["1st","2nd"])

MultiIndex([('A', 'x'),
            ('A', 'y'),
            ('B', 'x'),
            ('B', 'y'),
            ('C', 'x'),
            ('C', 'y')],
           names=['1st', '2nd'])

In [130]:
pd.MultiIndex.from_product([["A","B","C"],["x","y"]], names=["1st","2nd"])


MultiIndex([('A', 'x'),
            ('A', 'y'),
            ('B', 'x'),
            ('B', 'y'),
            ('C', 'x'),
            ('C', 'y')],
           names=['1st', '2nd'])

In [131]:
idx = pd.MultiIndex.from_tuples(tuples, names=["1st","2nd"])
pd.DataFrame(np.random.rand(6,2), index = idx, columns=["m","n"])

m         n
1st 2nd                    
A   x    0.599986  0.083346
    y    0.587151  0.969253
B   x    0.374709  0.274591
    y    0.291244  0.709574
C   x    0.410727  0.030833
    y    0.758743  0.899585

In [33]:
cidx = pd.MultiIndex.from_product([["A","B"],["x","y"]], names=["c1","c2"])
pd.DataFrame(np.random.rand(4,4), columns = cidx)

c1         A                   B          
c2         x         y         x         y
0   0.572727  0.324521  0.113746  0.733312
1   0.171528  0.763045  0.652714  0.633551
2   0.095320  0.072354  0.229110  0.513907
3   0.300453  0.919104  0.501841  0.900736

In [133]:
ridx = pd.MultiIndex.from_product([["m","n"],["l","p"]], names=["r1","r2"])
data = pd.DataFrame(np.random.rand(4,4), index= ridx, columns = cidx)

In [134]:
data

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  l   0.010072  0.752788  0.151848  0.498704
   p   0.454387  0.151825  0.273225  0.746636
n  l   0.403967  0.950200  0.297730  0.428465
   p   0.026189  0.735204  0.993310  0.713510

In [135]:
data["A"]

c2            x         y
r1 r2                    
m  l   0.010072  0.752788
   p   0.454387  0.151825
n  l   0.403967  0.950200
   p   0.026189  0.735204

In [136]:
data["x"] # This will return a KeyError: 'x'

KeyError: 'x'

In [137]:
data["m"]# the same error as in the previous cell KeyError: 'm'

KeyError: 'm'

In [ ]:
data["m", "A"] # the same here

In [139]:
data["A", "x"] # Here is the right way to index a Dataframe with Multiindexes

r1  r2
m   l     0.010072
    p     0.454387
n   l     0.403967
    p     0.026189
Name: (A, x), dtype: float64

In [140]:
data["A"]["x"] # Or this 

r1  r2
m   l     0.010072
    p     0.454387
n   l     0.403967
    p     0.026189
Name: x, dtype: float64

## MultiIndex: indexing via iloc

In [141]:
data 

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  l   0.010072  0.752788  0.151848  0.498704
   p   0.454387  0.151825  0.273225  0.746636
n  l   0.403967  0.950200  0.297730  0.428465
   p   0.026189  0.735204  0.993310  0.713510

In [142]:
data.iloc[0]

c1  c2
A   x     0.010072
    y     0.752788
B   x     0.151848
    y     0.498704
Name: (m, l), dtype: float64

In [143]:
data.iloc[(0,1)]

0.7527875105209206

In [144]:
data.iloc[[0,1]]

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  l   0.010072  0.752788  0.151848  0.498704
   p   0.454387  0.151825  0.273225  0.746636

In [145]:
data.loc[("m","l")]

c1  c2
A   x     0.010072
    y     0.752788
B   x     0.151848
    y     0.498704
Name: (m, l), dtype: float64

In [147]:
data.loc[:,("A","y")]

r1  r2
m   l     0.752788
    p     0.151825
n   l     0.950200
    p     0.735204
Name: (A, y), dtype: float64

In [148]:
data.loc["m":"n"]

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  l   0.010072  0.752788  0.151848  0.498704
   p   0.454387  0.151825  0.273225  0.746636
n  l   0.403967  0.950200  0.297730  0.428465
   p   0.026189  0.735204  0.993310  0.713510

In [149]:
data.loc[("m","l"):("n","l")]

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  l   0.010072  0.752788  0.151848  0.498704
   p   0.454387  0.151825  0.273225  0.746636
n  l   0.403967  0.950200  0.297730  0.428465

In [150]:
data.loc[("m","p"):"n"]

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  p   0.454387  0.151825  0.273225  0.746636
n  l   0.403967  0.950200  0.297730  0.428465
   p   0.026189  0.735204  0.993310  0.713510

In [151]:
data.loc[[("m","p"),("n","l")]]

c1            A                   B          
c2            x         y         x         y
r1 r2                                        
m  p   0.454387  0.151825  0.273225  0.746636
n  l   0.403967  0.950200  0.297730  0.428465

In [152]:
data.xs("p", level="r2")

c1         A                   B          
c2         x         y         x         y
r1                                        
m   0.454387  0.151825  0.273225  0.746636
n   0.026189  0.735204  0.993310  0.713510

In [153]:
data.xs("m", level="r1")

c1         A                   B          
c2         x         y         x         y
r2                                        
l   0.010072  0.752788  0.151848  0.498704
p   0.454387  0.151825  0.273225  0.746636

In [154]:
data.xs("y", level="c2", axis=1)

c1            A         B
r1 r2                    
m  l   0.752788  0.498704
   p   0.151825  0.746636
n  l   0.950200  0.428465
   p   0.735204  0.713510

In [155]:
data.xs("B", level="c1", axis=1)

c2            x         y
r1 r2                    
m  l   0.151848  0.498704
   p   0.273225  0.746636
n  l   0.297730  0.428465
   p   0.993310  0.713510

In [156]:
data_multiindex = pd.DataFrame([["bar", "one", "z", 1], 
                    ["bar", "two", "y", 2], 
                    ["foo", "one", "x", 3], 
                    ["foo", "two", "w", 4]], columns=['a', 'b', 'c', 'd'])
data_multiindex

,a,b,c,d
0,bar,one,z,1
1,bar,two,y,2
2,foo,one,x,3
3,foo,two,w,4


In [158]:
data_multiindex.set_index(['a','b'])

c  d
a   b        
bar one  z  1
    two  y  2
foo one  x  3
    two  w  4

In [159]:
data_multiindex.set_index('c', append=True)

,,a,b,d
,c,,,
0,z,bar,one,1
1,y,bar,two,2
2,x,foo,one,3
3,w,foo,two,4


In [160]:
data_multiindex.set_index(['a','b']).reset_index()


,a,b,c,d
0,bar,one,z,1
1,bar,two,y,2
2,foo,one,x,3
3,foo,two,w,4


In [161]:
data_multiindex.set_index(['a','b']).reset_index(level=1)

,b,c,d
a,,,
bar,one,z,1
bar,two,y,2
foo,one,x,3
foo,two,w,4


## Reshaping data

In [14]:
df = pd.read_csv("../data/reshaping.csv", index_col=0)
df

,country,year,type,count
0,A,1999,cases,0.7K
1,A,1999,pop,19M
2,A,2000,cases,2K
3,A,2000,pop,20M
4,B,1999,cases,37K
5,B,1999,pop,172M
6,B,2000,cases,80K
7,B,2000,pop,174M
8,C,1999,cases,212K
9,C,1999,pop,1T


In [15]:
df_wide = df.pivot(
  index=["country","year"], 
  columns="type", 
  values="count"
)
df_wide

type         cases   pop
country year            
A       1999  0.7K   19M
        2000    2K   20M
B       1999   37K  172M
        2000   80K  174M
C       1999  212K    1T
        2000  213K    1T

In [16]:
df_wide.index

MultiIndex([('A', 1999),
            ('A', 2000),
            ('B', 1999),
            ('B', 2000),
            ('C', 1999),
            ('C', 2000)],
           names=['country', 'year'])

In [17]:
df_wide.columns

Index(['cases', 'pop'], dtype='object', name='type')

In [18]:
df_wide.reset_index().rename_axis(columns=None)

,country,year,cases,pop
0,A,1999,0.7K,19M
1,A,2000,2K,20M
2,B,1999,37K,172M
3,B,2000,80K,174M
4,C,1999,212K,1T
5,C,2000,213K,1T


In [23]:
df = pd.read_csv("../data/rate.csv", index_col=0)
df

,country,year,rate
0,A,1999,0.7K/19M
1,A,2000,2K/20M
2,B,1999,37K/172M
3,B,2000,80K/174M
4,C,1999,212K/1T
5,C,2000,213K/1T


In [25]:
df.assign(rate = lambda d: d.rate.str.split("/"))

,country,year,rate
0,A,1999,"[0.7K, 19M]"
1,A,2000,"[2K, 20M]"
2,B,1999,"[37K, 172M]"
3,B,2000,"[80K, 174M]"
4,C,1999,"[212K, 1T]"
5,C,2000,"[213K, 1T]"


In [26]:
( df
  .assign(
    rate = lambda d: d.rate.str.split("/")
  )
  .explode("rate")
  .assign(
    type = lambda d: ["cases", "pop"] * int(d.shape[0]/2)
  )
  .pivot(index=["country","year"], columns="type", values="rate")
  .reset_index()
)

type,country,year,cases,pop
0,A,1999,0.7K,19M
1,A,2000,2K,20M
2,B,1999,37K,172M
3,B,2000,80K,174M
4,C,1999,212K,1T
5,C,2000,213K,1T


In [27]:
( df
  .assign(
    rate = lambda d: d.rate.str.split("/")
  )
  .explode("rate")
  .assign(
    type = lambda d: ["cases", "pop"] * int(d.shape[0]/2)
  )
)

,country,year,rate,type
0,A,1999,0.7K,cases
0,A,1999,19M,pop
1,A,2000,2K,cases
1,A,2000,20M,pop
2,B,1999,37K,cases
2,B,1999,172M,pop
3,B,2000,80K,cases
3,B,2000,174M,pop
4,C,1999,212K,cases
4,C,1999,1T,pop


## Split-Apply-Combine

In [36]:
cereal = pd.read_csv("../data/cereal.csv")
cereal

,name,mfr,type,calories,sugars,rating
0,100% Bran,Nabisco,Cold,70,6,68.402973
1,100% Natural Bran,Quaker Oats,Cold,120,8,33.983679
2,All-Bran,Kellogg's,Cold,70,5,59.425505
3,All-Bran with Extra Fiber,Kellogg's,Cold,50,0,93.704912
4,Almond Delight,Ralston Purina,Cold,110,8,34.384843
...,...,...,...,...,...,...
72,Triples,General Mills,Cold,110,3,39.106174
73,Trix,General Mills,Cold,110,12,27.753301
74,Wheat Chex,Ralston Purina,Cold,100,3,49.787445
75,Wheaties,General Mills,Cold,100,3,51.592193


In [37]:
cereal.groupby("type")

In [38]:
cereal 

,name,mfr,type,calories,sugars,rating
0,100% Bran,Nabisco,Cold,70,6,68.402973
1,100% Natural Bran,Quaker Oats,Cold,120,8,33.983679
2,All-Bran,Kellogg's,Cold,70,5,59.425505
3,All-Bran with Extra Fiber,Kellogg's,Cold,50,0,93.704912
4,Almond Delight,Ralston Purina,Cold,110,8,34.384843
...,...,...,...,...,...,...
72,Triples,General Mills,Cold,110,3,39.106174
73,Trix,General Mills,Cold,110,12,27.753301
74,Wheat Chex,Ralston Purina,Cold,100,3,49.787445
75,Wheaties,General Mills,Cold,100,3,51.592193


In [39]:
cereal.groupby("type").groups

{'Cold': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76], 'Hot': [20, 43, 57]}

In [45]:
gp = cereal.groupby(["type"])

In [55]:
dir(gp)

['_DataFrameGroupBy__examples_dataframe_doc',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_agg_examples_doc',
 '_agg_general',
 '_agg_py_fallback',
 '_aggregate_frame',
 '_aggregate_with_numba',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_ascending_count',
 '_bool_agg',
 '_cache',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_general',
 '_cython_transform',
 '_define_paths',
 '_descending_count',
 '_dir_additions',
 '_dir_deletions',
 '

## GroupBy dropna
By default NA values are excluded from group keys during the groupby operation. However, in case you want to include NA values in group keys, you could pass dropna=False to achieve it.

In [73]:
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]

df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])

df_dropna

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [74]:
df_dropna.groupby(by=["b"], dropna=True).sum()

,a,c
b,,
1.0,2,3
2.0,2,5


In [76]:
df_false = pd.DataFrame(range(4), index=["a","a","a","a"])

In [84]:
df_false[0]["a"]

a    0
a    1
a    2
a    3
Name: 0, dtype: int64

In [85]:
df_false = pd.DataFrame(range(4), index=["a","a","a","b"])

In [87]:
df_false[0]["b"]

3

In [163]:
gp.mean()

TypeError: Could not convert 100% Bran100% Natural BranAll-BranAll-Bran with Extra FiberAlmond DelightApple Cinnamon CheeriosApple JacksBasic 4Bran ChexBran FlakesCap'n'CrunchCheeriosCinnamon Toast CrunchClustersCocoa PuffsCorn ChexCorn FlakesCorn PopsCount ChoculaCracklin' Oat BranCrispixCrispy Wheat & RaisinsDouble ChexFroot LoopsFrosted FlakesFrosted Mini-WheatsFruit & Fibre Dates; Walnuts; and OatsFruitful BranFruity PebblesGolden CrispGolden GrahamsGrape Nuts FlakesGrape-NutsGreat Grains PecanHoney Graham OhsHoney Nut CheeriosHoney-combJust Right Crunchy  NuggetsJust Right Fruit & NutKixLifeLucky CharmsMuesli Raisins; Dates; & AlmondsMuesli Raisins; Peaches; & PecansMueslix Crispy BlendMulti-Grain CheeriosNut&Honey CrunchNutri-Grain Almond-RaisinNutri-grain WheatOatmeal Raisin CrispPost Nat. Raisin BranProduct 19Puffed RicePuffed WheatQuaker Oat SquaresRaisin BranRaisin Nut BranRaisin SquaresRice ChexRice KrispiesShredded WheatShredded Wheat 'n'BranShredded Wheat spoon sizeSmacksS

In [162]:
gp.mean? # Run this to check the doc and notice by using numeric_only=True, you overcome the previous error message.

In [165]:
gp.mean(numeric_only = True)

,calories,sugars,rating
type,,,
Cold,107.162162,7.175676,42.095218
Hot,100.000000,1.333333,56.737708


In [60]:
gp["calories"].mean()

type
Cold    107.162162
Hot     100.000000
Name: calories, dtype: float64

In [63]:
gp[["calories", "rating", "sugars"]].mean()

,calories,rating,sugars
type,,,
Cold,107.162162,42.095218,7.175676
Hot,100.000000,56.737708,1.333333


In [64]:
cereal.groupby("mfr").size()

mfr
General Mills     22
Kellogg's         23
Maltex             1
Nabisco            6
Post               9
Quaker Oats        8
Ralston Purina     8
dtype: int64

In [65]:
cereal.groupby("type").get_group("Hot")

,name,mfr,type,calories,sugars,rating
20,Cream of Wheat (Quick),Nabisco,Hot,100,0,64.533816
43,Maypo,Maltex,Hot,100,3,54.850917
57,Quaker Oatmeal,Quaker Oats,Hot,100,1,50.828392


In [66]:
cereal.groupby("mfr").get_group("Post")

,name,mfr,type,calories,sugars,rating
9,Bran Flakes,Post,Cold,90,5,53.313813
27,Fruit & Fibre Dates; Walnuts; and Oats,Post,Cold,120,10,40.917047
29,Fruity Pebbles,Post,Cold,110,12,28.025765
30,Golden Crisp,Post,Cold,100,15,35.252444
32,Grape Nuts Flakes,Post,Cold,100,5,52.076897
33,Grape-Nuts,Post,Cold,110,3,53.371007
34,Great Grains Pecan,Post,Cold,120,4,45.811716
37,Honey-comb,Post,Cold,110,11,28.742414
52,Post Nat. Raisin Bran,Post,Cold,120,14,37.840594


In [68]:
for name, group in cereal.groupby("type"):
    print(name)
    print(group)
    print("")

Cold
                         name             mfr  type  calories  sugars   
0                   100% Bran         Nabisco  Cold        70       6  \
1           100% Natural Bran     Quaker Oats  Cold       120       8   
2                    All-Bran       Kellogg's  Cold        70       5   
3   All-Bran with Extra Fiber       Kellogg's  Cold        50       0   
4              Almond Delight  Ralston Purina  Cold       110       8   
..                        ...             ...   ...       ...     ...   
72                    Triples   General Mills  Cold       110       3   
73                       Trix   General Mills  Cold       110      12   
74                 Wheat Chex  Ralston Purina  Cold       100       3   
75                   Wheaties   General Mills  Cold       100       3   
76        Wheaties Honey Gold   General Mills  Cold       110       8   

       rating  
0   68.402973  
1   33.983679  
2   59.425505  
3   93.704912  
4   34.384843  
..        ...  
72  39

In [69]:
cereal.groupby("mfr", as_index=False).agg(
min_cal = ("calories", "min"),
max_cal = ("calories", "max"),
med_sugar = ("sugars", "median"),
avg_rating = ("rating", "mean"))

,mfr,min_cal,max_cal,med_sugar,avg_rating
0,General Mills,100,140,8.5,34.485852
1,Kellogg's,50,160,7.0,44.038462
2,Maltex,100,100,3.0,54.850917
3,Nabisco,70,100,0.0,67.968567
4,Post,90,120,10.0,41.705744
5,Quaker Oats,50,120,6.0,42.915990
6,Ralston Purina,90,150,5.5,41.542997


In [92]:
cereal.groupby("mfr", as_index=False).agg(
min_cal = pd.NamedAgg("calories", "min"),
max_cal = ("calories", "max"),
med_sugar = ("sugars", "median"),
avg_rating = ("rating", "mean"))

,mfr,min_cal,max_cal,med_sugar,avg_rating
0,General Mills,100,140,8.5,34.485852
1,Kellogg's,50,160,7.0,44.038462
2,Maltex,100,100,3.0,54.850917
3,Nabisco,70,100,0.0,67.968567
4,Post,90,120,10.0,41.705744
5,Quaker Oats,50,120,6.0,42.915990
6,Ralston Purina,90,150,5.5,41.542997


In [72]:
cereal.groupby("type").transform(np.mean)

TypeError: Could not convert 100% Bran100% Natural BranAll-BranAll-Bran with Extra FiberAlmond DelightApple Cinnamon CheeriosApple JacksBasic 4Bran ChexBran FlakesCap'n'CrunchCheeriosCinnamon Toast CrunchClustersCocoa PuffsCorn ChexCorn FlakesCorn PopsCount ChoculaCracklin' Oat BranCrispixCrispy Wheat & RaisinsDouble ChexFroot LoopsFrosted FlakesFrosted Mini-WheatsFruit & Fibre Dates; Walnuts; and OatsFruitful BranFruity PebblesGolden CrispGolden GrahamsGrape Nuts FlakesGrape-NutsGreat Grains PecanHoney Graham OhsHoney Nut CheeriosHoney-combJust Right Crunchy  NuggetsJust Right Fruit & NutKixLifeLucky CharmsMuesli Raisins; Dates; & AlmondsMuesli Raisins; Peaches; & PecansMueslix Crispy BlendMulti-Grain CheeriosNut&Honey CrunchNutri-Grain Almond-RaisinNutri-grain WheatOatmeal Raisin CrispPost Nat. Raisin BranProduct 19Puffed RicePuffed WheatQuaker Oat SquaresRaisin BranRaisin Nut BranRaisin SquaresRice ChexRice KrispiesShredded WheatShredded Wheat 'n'BranShredded Wheat spoon sizeSmacksS

In [166]:
cereal.groupby("type").transform(np.mean, numeric_only=True)

,calories,sugars,rating
0,107.162162,7.175676,42.095218
1,107.162162,7.175676,42.095218
2,107.162162,7.175676,42.095218
3,107.162162,7.175676,42.095218
4,107.162162,7.175676,42.095218
...,...,...,...
72,107.162162,7.175676,42.095218
73,107.162162,7.175676,42.095218
74,107.162162,7.175676,42.095218
75,107.162162,7.175676,42.095218
